# ऑटोजेन के साथ एजेंटिक RAG

यह नोटबुक ऑटोजेन एजेंट्स का उपयोग करके रिट्रीवल-ऑगमेंटेड जनरेशन (RAG) को लागू करने और उन्नत मूल्यांकन क्षमताओं के साथ प्रदर्शित करती है।


SQLite संस्करण सुधार  
यदि आपको यह त्रुटि मिले:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

अपने नोटबुक की शुरुआत में इस कोड ब्लॉक को अनकमेंट करें:  


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## क्लाइंट बनाएं

सबसे पहले, हम Azure AI Chat Completion क्लाइंट को इनिशियलाइज़ करते हैं। इस क्लाइंट का उपयोग Azure OpenAI सेवा के साथ इंटरैक्ट करने और उपयोगकर्ता के प्रश्नों के उत्तर उत्पन्न करने के लिए किया जाएगा।


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## वेक्टर डेटाबेस प्रारंभिकरण

हम ChromaDB को स्थायी स्टोरेज के साथ प्रारंभ करते हैं और उन्नत नमूना दस्तावेज़ जोड़ते हैं। ChromaDB का उपयोग उन दस्तावेज़ों को संग्रहीत और पुनः प्राप्त करने के लिए किया जाएगा जो सटीक उत्तर उत्पन्न करने के लिए संदर्भ प्रदान करते हैं।


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## संदर्भ प्रदाता कार्यान्वयन

`ContextProvider` क्लास कई स्रोतों से संदर्भ प्राप्त करने और एकीकृत करने का कार्य करता है:

1. **वेक्टर डेटाबेस पुनःप्राप्ति**: यात्रा दस्तावेज़ों पर सेमांटिक सर्च करने के लिए ChromaDB का उपयोग करता है  
2. **मौसम जानकारी**: प्रमुख शहरों के लिए एक सिम्युलेटेड मौसम डेटाबेस बनाए रखता है  
3. **एकीकृत संदर्भ**: दस्तावेज़ और मौसम डेटा दोनों को मिलाकर व्यापक संदर्भ प्रदान करता है  

मुख्य विधियाँ:
- `get_retrieval_context()`: क्वेरी के आधार पर प्रासंगिक दस्तावेज़ प्राप्त करता है  
- `get_weather_data()`: निर्दिष्ट स्थानों के लिए मौसम की जानकारी प्रदान करता है  
- `get_unified_context()`: बेहतर प्रतिक्रियाओं के लिए दस्तावेज़ और मौसम संदर्भ दोनों को जोड़ता है  


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## एजेंट कॉन्फ़िगरेशन

हम पुनः प्राप्ति और सहायक एजेंट्स को कॉन्फ़िगर करते हैं। पुनः प्राप्ति एजेंट प्रासंगिक जानकारी खोजने में विशेषज्ञ है, जो सेमांटिक सर्च का उपयोग करता है, जबकि सहायक एजेंट पुनः प्राप्त जानकारी के आधार पर विस्तृत उत्तर तैयार करता है।


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## RAG के साथ क्वेरी प्रोसेसिंग

हम `ask_rag` फ़ंक्शन को परिभाषित करते हैं ताकि क्वेरी को सहायक को भेजा जा सके, प्रतिक्रिया को प्रोसेस किया जा सके, और उसका मूल्यांकन किया जा सके। यह फ़ंक्शन सहायक के साथ इंटरैक्शन को संभालता है और प्रतिक्रिया की गुणवत्ता को मापने के लिए मूल्यांकनकर्ता का उपयोग करता है।


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# उदाहरण उपयोग

हम मूल्यांकनकर्ता को प्रारंभ करते हैं और उन प्रश्नों को परिभाषित करते हैं जिन्हें हम संसाधित और मूल्यांकन करना चाहते हैं।


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## स्क्रिप्ट चलाएं

हम जांचते हैं कि स्क्रिप्ट एक इंटरएक्टिव वातावरण में चल रही है या एक सामान्य स्क्रिप्ट में, और मुख्य फ़ंक्शन को उसी के अनुसार चलाते हैं।


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
